### 0. Prepare the dataset for the subsequent modelling.
1. Download the heart disease dataset from https://www.statlearning.com/s/Heart.csv
2. Load the dataset and drop all variables except the predictors Age, Sex, ChestPain, RestBP, Chol, and the target variable AHD. Drop all rows containing a NaN value.
3. Onehot encode the variable ChestPain. This means that where you before had a single column with one of four values ['typical', 'asymptomatic', 'nonanginal', 'nontypical'], you will now have four binary columns (their names don't matter), akin to 'ChestPain_typical' 'ChestPain_asymptomatic', 'ChestPain_nonanginal', 'ChestPain_nontypical'. A row that before had ChestPain='typical' will now have ChestPain_typical=1 and the other three columns set to 0, ChestPain='asymptomatic' will have ChestPain_asymptomatic=1 and the other three set to 0, etc.
4. Binary encode the target variable AHD such that 'No'=0 and 'Yes'=1.

In [1]:
import pandas as pd


predictors = ['Age', 'Sex', 'ChestPain', 'RestBP', 'Chol']
target = 'AHD'

df = pd.read_csv('https://www.statlearning.com/s/Heart.csv')
df = df[predictors + [target]]
df = df.dropna()
df = pd.get_dummies(df, columns=['ChestPain'])
df['AHD'] = df['AHD'].map({'No': 0, 'Yes': 1})
df.head()

,Age,Sex,RestBP,Chol,AHD,ChestPain_asymptomatic,ChestPain_nonanginal,ChestPain_nontypical,ChestPain_typical
0,63,1,145,233,0,False,False,False,True
1,67,1,160,286,1,True,False,False,False
2,67,1,120,229,1,True,False,False,False
3,37,1,130,250,0,False,True,False,False
4,41,0,130,204,0,False,False,True,False


### 1. Fit a model using a standard train/validation split through multiple steps.
Through the steps you will practice chaining functions, and you will also create the infrastructure necessary for the remaining tasks.

1. Write a function "stratified_split" that takes three arguments: A dataframe, a number of folds, and a list of variables to stratify by. The function should return a list of dataframes, one for each fold, where the dataframes are stratified by the variables in the list. Test that the function works by splitting the dataset into two folds based on 'AHD', 'Age' and 'RestBP' and print the size of each fold, the counts of 0s and 1s in AHD, and the mean of each of 'Age' and 'RestBP' (all these should be printed individually per fold). Ensure that the function does not modify the original dataframe.

In [6]:
import numpy as np
from collections import Counter


def stratified_split(dataframe: pd.DataFrame, num_folds: int, variables):
    dataframe = dataframe.copy()

    dataframe = dataframe.sort_values(variables)
    dataframe['fold'] = np.arange(len(dataframe)) % num_folds

    return [dataframe[dataframe['fold'] == fold] for fold in range(num_folds)]

folds = stratified_split(df, 2, ['AHD', 'Age', 'RestBP'])

for fold in folds:
    print(len(fold))
    print(Counter(fold['AHD']))

152
Counter({0: 82, 1: 70})
151
Counter({0: 82, 1: 69})


2. Write a function 'fit_and_predict' that takes 4 arguments: A training set, a validation set, a list of predictors, and a target variable. The function should fit a logistic regression model to the training set using the predictors and target variable, and return the predictions of the model on the validation set.

In [13]:
from sklearn.linear_model import LogisticRegression


def fit_and_predict(train, validation, predictors, target):
    model = LogisticRegression()
    model.fit(train[predictors], train[target])

    return model.predict_proba(validation[predictors])[:, 1]

3. Write a function 'fit_and_predict_standardized' that takes 5 arguments: A training set, a validation set, a list of predictors, a target variable, and a list of variables to standardize. Using a loop (or a scaler), the function should z-score standardize the given variables in both the training set and the validation set based on the mean and standard deviation in the training set. Then, the function should call the 'fit_and_predict' function and return its result. Ensure that the function does not modify the original dataframes. Test the function using the train and validation set from above (e.g. the two folds from the split), while standardizing the 'Age', 'RestBP' and 'Chol' variables (as mentioned above, the target should be AHD, and you should also include the remaining predictors: 'Sex' and the ChestPain-variables)

In [14]:
from sklearn.preprocessing import StandardScaler


def fit_and_predict_standardized(train, validation, predictors, target, variables):
    scaler = StandardScaler()
    train[variables] = scaler.fit_transform(train[variables])
    validation[variables] = scaler.transform(validation[variables])

    return fit_and_predict(train, validation, predictors, target)

predictors = [predictor for predictor in predictors if predictor != 'ChestPain']
predictors += [col for col in df if col.startswith('ChestPain')]

fit_and_predict_standardized(folds[0], folds[1], predictors, target, ['Age', 'RestBP', 'Chol'])

array([0.25106997, 0.0896915 , 0.04101341, 0.1866407 , 0.05234733,
       0.35615682, 0.10848761, 0.04664008, 0.03054168, 0.19979049,
       0.26904313, 0.12079627, 0.20996988, 0.39417837, 0.64992003,
       0.2130914 , 0.05432808, 0.13362936, 0.2120067 , 0.14496902,
       0.65906444, 0.67638455, 0.03950789, 0.12978099, 0.38239122,
       0.69722618, 0.26697454, 0.25495893, 0.17270218, 0.40331776,
       0.3774143 , 0.04705159, 0.23981485, 0.26755535, 0.46755347,
       0.08885281, 0.78025424, 0.46815731, 0.2045457 , 0.09898492,
       0.53156096, 0.09738467, 0.45621057, 0.47470795, 0.19090526,
       0.08929871, 0.32157531, 0.10550725, 0.37101755, 0.23233461,
       0.06807867, 0.52548927, 0.24400157, 0.79497004, 0.50310015,
       0.83301662, 0.41597326, 0.33410234, 0.25638556, 0.40350471,
       0.8405122 , 0.2918181 , 0.67390036, 0.12925751, 0.46135751,
       0.30533789, 0.58834899, 0.1061003 , 0.60913318, 0.59985739,
       0.71887418, 0.8669327 , 0.20239776, 0.86937149, 0.35753

4. Write a function 'fit_and_compute_auc' that takes 5 arguments: A training set, a validation set, a list of predictors, a target variable, and a list of variables to standardize. The function should call the 'fit_and_predict_standardized' function to retrieve out-of-sample predictions for the validation set. Based on these and the ground truth labels in the validation set, it should compute and return the AUC. Test the function using the train and test set from above, while standardizing the 'Age', 'RestBP' and 'Chol' variables (and including the remaining predictors). Print the AUC.

In [16]:
from sklearn.metrics import roc_auc_score

def fit_and_compute_auc(train, validation, predictors, target, variables):
    predictions = fit_and_predict_standardized(train, validation, predictors, target, variables)

    return roc_auc_score(validation[target], predictions)

fit_and_compute_auc(folds[0], folds[1], predictors, target, ['Age', 'RestBP', 'Chol'])

0.8405797101449276

### 2. Perform a cross-validation.
Use the 'stratified_split' function to split the dataset into 10 folds, stratified on variables you find reasonable. For each fold, use the 'fit_and_compute_auc' function to compute the AUC of the model on the held-out validation set. Print the mean and standard deviation of the AUCs across the 10 folds.

In [20]:
folds = stratified_split(df, 10, ['AHD', 'Age', 'RestBP'])

aucs = []

for i in range(len(folds)):
    validation_fold = folds[i]
    train_fold = pd.concat([fold for j, fold in enumerate(folds) if i != j])

    aucs.append(fit_and_compute_auc(train_fold, validation_fold, predictors, target, ['Age']))

print(aucs)

[0.8403361344537814, 0.773109243697479, 0.7983193277310925, 0.755656108597285, 0.875, 0.9419642857142857, 0.8616071428571428, 0.7589285714285714, 0.7410714285714286, 0.8348214285714286]


/Users/esten/miniconda3/envs/ml/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/esten/miniconda3/envs/ml/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

### 3. Use the bootstrap to achieve a distribution of out-of-bag AUCs.
For 100 iterations, create a bootstrap sample by sampling with replacement from the full dataset until you have a training set equal in size to 80% of the original data. Use the observations not included in the bootstrap sample as the validation set for that iteration.. Fit models and calculate AUCs for each iteration. Print the mean and standard deviation of the AUCs.

### 4. Theory
1. List some benefits of wrapping code in functions rather than copying and pasting it multiple times.

- _Code can be reused in multiple different scenarios, meaning you have to write less code_
- _If you discover a bug in the code, that bug only has to be fixed in one place_
- _You hide complex implementation details behind a more abstract interface, making it easier to separate concerns_

2.  Explain three classification metrics and their benefits and drawbacks.

- _Logloss: Has nice mathematical properties that allows for using it as a loss function when fitting models. Hard to interpret_
- _Accuracy: Very intuitive to understand (the proportion of correct predictions). Does not take into account the cost of different misclassifications, __does not handle imbalanced classes__._
- _Area under the receiver operating characteristic curve (AUC/AUROC): Does not rely on setting a classification threshold, handles class imbalance. Hard to interpret literally (e.g. an AUC of 0.95 is generally good, but what does it mean more concretely?), can't be used for optimizing models_

3. Write a couple of sentences comparing the three methods (train/validation, cross-validation, bootstrap) above as approaches to quantify model performance. Which one yielded the best results? Which one would you expect to yield the best results? Can you mention some theoretical benefits and drawbacks with each? Even if you didn't do the optional bootstrap exercise you should reflect on this as an approach.

In [ ]:
print(f'Train/validation-split AUC: {split_auc:.2f}')
print(f'Cross-validation AUC: {np.mean(cv_aucs):.2f}+/-{np.std(cv_aucs):.2f}')
print(f'Bootstrap AUC: {np.mean(bootstrap_aucs):.2f}+/-{np.std(bootstrap_aucs):.2f}')

_In our case, the three methods yield results that are statistically equivalent. This is what we would expect, but given the fact that the single train/validation-split has a lot of variance depending on the exact split, this doesn't always happen. As such, one of the two latter is preferable_

4. Why do we stratify the dataset before splitting?

_To ensure the different folds of the dataset are similar with respect to certain key variables. If we don't do this, we could arrive at models that are very good in whatever portion they are trained on but very poor in everything else, simply due to the training population not being representative for the rest of the data._

5. What other use cases can you think of for the bootstrap method?

_In addition to assessing model performance, the bootstrap can also be used to get an idea of the spread of estimated parameter values (e.g. how important a variable is for a prediction)._